<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/code/SV/5s/SV_fcn_projection_guanape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Guanape

In [13]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
%cd drive/My\ Drive/

[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *

## Load model

In [16]:
class FCNet(nn.Module):

    def __init__(self):
        super(FCNet, self).__init__()

        self.cnn = nn.Sequential(
            nn.Linear(21*3,100),
            nn.ReLU(),
            nn.Linear(100,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,500),
            nn.ReLU(),
            nn.Linear(500,1)
        )

    def forward(self, x):
        out = x.reshape((x.size()[0], 1, 21*3))
        out = self.cnn(out)
        return out

def get_score(out, y):
    out, y = out.cpu(), y.cpu()
    out = 1*(out>0)
    true_positive = np.mean(out[y == True].numpy()) 
    true_negative = 1-np.mean(out[y == False].numpy())
    
    return (round(true_positive*100) , round(true_negative*100))

In [17]:
model = torch.load('SV_fcn_lonlatcov_5s.pt')
window = 21
variable = ('lon_std', 'lat_std', 'gaps')

# Test

In [18]:
data_train = pd.read_csv('./data/SV_train.csv')
data_validation = pd.read_csv('./data/SV_validation.csv')
data_test = pd.read_csv('./data/SV_test.csv')
data_test_guanape = pd.read_csv('./data/SV_test_guanape.csv')

data = pd.concat([data_train, data_validation, data_test, data_test_guanape])

In [19]:
resolution = 5
data_new = change_resolution(data, resolution)
data_new = standardize_data(data_new)

data_new.head()

,trip,datetime,lon,lat,gaps,dive,step_speed,step_direction,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
2,P1108_46_SV_T4,2008-12-06 16:06:25,-77.265897,-11.774297,0.0,0,15.301035,26.025648,2.309584,0.788287,0.826924,0.898598,0.438773
3,P1108_46_SV_T4,2008-12-06 16:06:30,-77.266435,-11.774462,0.0,0,12.287117,-18.228550,2.293957,0.776022,0.663327,0.949816,-0.312808
4,P1108_46_SV_T4,2008-12-06 16:06:35,-77.266843,-11.774868,0.0,0,12.679214,-28.073536,2.282105,0.745841,0.684610,0.882344,-0.470604
5,P1108_46_SV_T4,2008-12-06 16:06:40,-77.267200,-11.775312,0.0,0,12.579325,-6.324066,2.271735,0.712835,0.679188,0.993915,-0.110152
6,P1108_46_SV_T4,2008-12-06 16:06:45,-77.267390,-11.775845,0.0,0,12.567659,-18.969971,2.266216,0.673214,0.678555,0.945689,-0.325073


In [22]:
dive_estim = []

for i in data_new.trip.unique():
    # create dataset for a trajectory
    t = data_new[data_new.trip == i].copy()    
    test_set = TrajDataSet(t, window, variable, transform = transforms.Compose([Center(), ToTensor()]))

    model.eval()
    estim = [0 for i in range(10)]
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x)
            
            estim.append(out.cpu().squeeze().numpy())

    estim = estim + [0 for i in range(10)]
    dive_estim.append(estim)

    print(i)
data_new['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

P1108_46_SV_T4
P1108_4_SV_T1
P1108_4_SV_T2
P1108_6_SV_T1
P1109_21_SV_T4
P1109_21_SV_T7
P1111_27_SV_T2
P1111_27_SV_T4
P1111_41_SV_T1
P1111_41_SV_T4
P1111_46_SV_T5
P1111_9_SV_T2
P1112_10_SV_T1
P1112_11_SV_T7
P1112_12_SV_T1
P1112_12_SV_T4
P1112_14_SV_T1
P1112_14_SV_T2
P1112_15_SV_T1
P1112_15_SV_T2
P1112_15_SV_T3
P1112_15_SV_T4
P1112_18_SV_T2
P1112_18_SV_T3
P1112_18_SV_T5
P1112_19_SV_T2
P1112_19_SV_T3
P1112_19_SV_T4
P1112_19_SV_T5
P1112_19_SV_T6
P1112_19_SV_T7
P1112_20_SV_T1
P1112_20_SV_T2
P1112_20_SV_T3
P1112_21_SV_T1
P1112_21_SV_T2
P1112_22_SV_T1
P1112_22_SV_T2
P1112_22_SV_T3
P1112_24_SV_T1
P1112_24_SV_T3
P1112_24_SV_T4
P1112_25_SV_T1
P1112_25_SV_T3
P1112_25_SV_T4
P1112_25_SV_T5
P1112_26_SV_T3
P1112_27_SV_T1
P1112_27_SV_T2
P1112_29_SV_T2
P1112_29_SV_T3
P1112_30_SV_T1
P1112_30_SV_T2
P1112_31_SV_T1
P1112_31_SV_T2
P1112_31_SV_T5
P1112_33_SV_T2
P1112_34_SV_T1
P1112_35_SV_T2
P1112_37_SV_T1
P1112_37_SV_T2
P1112_37_SV_T3
P1112_38_SV_T1
P1112_38_SV_T2
P1112_3_SV_T2
P1112_4_SV_T2
P1112_4_SV_T4
P1

In [ ]:
data_test_new = data_new[[x[0] == 'G' for x in data_new.trip]]

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

# globally
TP = []
FP = []

for tt in np.arange(0,1,0.001):
    all_estim = 1* (data_test_new.prediction > tt)
    true_positive = np.mean(all_estim[data_test_new.dive == 1])
    true_negative = 1-np.mean(all_estim[data_test_new.dive == 0])
    TP.append(true_positive)
    FP.append(1-true_negative)

plt.plot(np.array(FP), np.array(TP))

# Export Probabilities

In [ ]:
data_new.to_csv('SV_fcn_projection.csv', index = False)